In [ ]:
import pandas as pd
IDSS = pd.read_excel('../dieter/wards.xlsx')
IDSS = list(IDSS['WARD_ID'])
col_names = [int(i) for i in IDSS]

In [ ]:
col_names1 = []
for i in col_names:
    j = str(i)[0:3]+str(i)[4:8]
    col_names1.append(j)
col_names1

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

In [ ]:
import ee
import os
import geemap
import pandas as pd

# Initialize the Earth Engine module.
#ee.Initialize()

def calculate_crop_area_npp(ward_shapefile_id, land_cover_dataset_id, npp_dataset_id, year):
    """
    Calculate crop area and extract NPP for a specific ward.

    Args:
    - ward_shapefile_id (str): Asset ID of the ward shapefile in GEE
    - land_cover_dataset_id (str): Asset ID of the SANLC dataset in GEE
    - npp_dataset_id (str): Asset ID of the MODIS NPP dataset in GEE
    - year (int): Year for NPP data extraction

    Returns:
    - dict: A dictionary with ward ID, crop area, and mean NPP
    """
    try:
        # Load the ward shapefile.
        ward = ee.FeatureCollection(ward_shapefile_id)

        # Load the land cover and NPP datasets.
        land_cover = ee.Image(land_cover_dataset_id)
        npp_dataset = ee.ImageCollection(npp_dataset_id).filter(ee.Filter.date(f'{year}-01-01', f'{year}-12-31')).mean()

        # Filter the land cover image for crop areas (assuming crop categories are known).
        crop_category_ids = [u for u in range(32,33)]
        crop_mask = land_cover.eq([crop_category_ids])  # Replace with actual crop category IDs.

        # Calculate the crop area in the ward.
        crop_area = crop_mask.multiply(ee.Image.pixelArea()).reduceRegion(
            reducer=ee.Reducer.sum(),
            geometry=ward.geometry(),
            scale=30,
            maxPixels=1e9
        ).get('classification', 0)

        # Calculate the mean NPP for the crop area in the ward.
        mean_npp = npp_dataset.updateMask(crop_mask).reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=ward.geometry(),
            scale=500,  # Adjust scale according to the resolution you desire
            maxPixels=1e9
        ).get('Npp', 0)
    except (Exception):
        print(f'Shapefile for ward {i} not found.')
        #crop_area = 0
        #mean_npp  = 0
    return {
        'Ward ID': ward_shapefile_id,
        'Crop Area (sq. m)': crop_area.getInfo(),
        'Mean NPP (gC/m^2/year)': mean_npp.getInfo()
    }

# Example usage for multiple wards
ward_ids =  [f'users/noecareme/ADM4_PCODE_ZA{i}' for i in col_names1[:1]]  # List of ward asset IDs
land_cover_dataset_id = 'users/noecareme/SA_NLC_2020_GEO'  # Replace with your SANLC dataset asset ID
npp_dataset_id = 'MODIS/006/MOD17A3HGF'  # MODIS NPP dataset
year = 2018  # Year of interest

results = []
for ward_id in ward_ids:
    result = calculate_crop_area_npp(ward_id, land_cover_dataset_id, npp_dataset_id, year)
    results.append(result)

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)
print(results_df)